In [98]:
import os
import mxnet as mx
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd
import copy # for deep copying models

In [2]:
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data import DataLoader
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from mxnet import gluon, nd, autograd

In [3]:
data_dir = "../../../data/data_generated_medicotask_70_30_v2" #main_data_dir

train_data_dir = f'{data_dir}/train'
validation_data_dir = f'{data_dir}/validation'

In [4]:
len(os.listdir(train_data_dir))

16

In [5]:
TARGET_SIZE = 224
SIZE = (TARGET_SIZE, TARGET_SIZE)
BATCH_SIZE = 32
NUM_WORKERS = 4#multiprocessing.cpu_count()
NUM_OF_EPOCHS = 1

In [7]:
model_name = "mxnet_optimization_test_1"
model_dir = data_dir + '/mxnet_models'
plot_dir  = data_dir + '/mxnet_plots'
history_dir = data_dir + '/mxnet_history'

acc_loss_plot_name = 'acc_loss_plot_' + model_name

train_data_dir = f'{data_dir}/train'
validation_data_dir = f'{data_dir}/validation'

In [8]:
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

if not os.path.exists(history_dir):
    os.mkdir(history_dir)


In [9]:
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()

In [10]:
def transform(image, label):
     # resize the shorter edge to 224, the longer edge will be greater or equal to 224
    resized = mx.image.resize_short(image, TARGET_SIZE)
    # center and crop an area of size (224,224)
    cropped, crop_info = mx.image.center_crop(resized, SIZE)
    #transpose the channels to be (3,224,224)
    transposed = nd.transpose(cropped, (2,0,1))
    return transposed, label

In [11]:
dataset_train = ImageFolderDataset(root=train_data_dir, transform=transform)
dataset_test = ImageFolderDataset(root=validation_data_dir, transform=transform)

In [12]:
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS) # last_batch='discard' (removed for testing)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, # last_batch='discard',
                             shuffle=True, num_workers=NUM_WORKERS)
print("Train dataset: {} images, Test dataset: {} images".format(len(dataset_train), len(dataset_test)))

Train dataset: 3697 images, Test dataset: 1596 images


In [13]:
categories = dataset_train.synsets
NUM_CLASSES = len(categories)

In [14]:
categories

['blurry-nothing',
 'colon-clear',
 'dyed-lifted-polyps',
 'dyed-resection-margins',
 'esophagitis',
 'instruments',
 'normal-cecum',
 'normal-pylorus',
 'normal-z-line',
 'out-of-patient',
 'polyps',
 'retroflex-rectum',
 'retroflex-stomach',
 'stool-inclusions',
 'stool-plenty',
 'ulcerative-colitis']

In [15]:
dataloader_train

## MOdel creation

In [16]:
# get pretrained squeezenet
net = models.resnet50_v2(pretrained=True, prefix='medico_task',ctx=ctx)

In [17]:
my_net = models.resnet50_v2(prefix='medico_task', classes=NUM_CLASSES, ctx=ctx)

In [18]:
my_net.collect_params().initialize(ctx=ctx)

In [19]:
my_net.features = net.features

In [20]:
len(dataset_test)

1596

In [67]:

#metric_loss = mx.metric.Loss()

def evaluate_accuracy(data_iterator,loss_fn, net):
    metric_acc = mx.metric.Accuracy()
  #  numerator = 0.
  #  denominator = 0.
    cumulative_loss = mx.nd.zeros(1, ctx=ctx)
    no_of_samples = mx.nd.zeros(1, ctx=ctx)

    for i, (data, label) in enumerate(data_iterator):
        with autograd.predict_mode():
            data = data.astype(np.float32).as_in_context(ctx)
            label = label.astype(np.int32).as_in_context(ctx)
            output = net(data)
            loss = loss_fn(output, label)
            prediction = nd.argmax(output, axis=1).astype(np.int32)
            cumulative_loss += nd.sum(loss).asscalar()
            no_of_samples += data.shape[0]

        metric_acc.update([label], [prediction])
        #metric_loss.update([label], [prediction])
    
    
    print("cumulative loss = {0} no_of_samples = {1}" .format(cumulative_loss, no_of_samples))
    loss = cumulative_loss / no_of_samples
    return metric_acc.get()[1], loss

In [68]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [70]:
out_acc, out_loss  = evaluate_accuracy(dataloader_test, softmax_cross_entropy, my_net)

cumulative loss = 
[344.77698]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>


In [71]:
out_acc

0.9266917293233082

In [99]:
out_loss.asscalar()

0.21602574

In [100]:
my_net.featu

HybridSequential(
  (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, use_global_stats=False, in_channels=3)
  (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
  (3): Activation(relu)
  (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
  (5): HybridSequential(
    (0): BottleneckV2(
      (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      (conv1): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn3): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
      (conv3): Conv2D

In [103]:
weights = my_net.params()[0]['"conv1"'].asnumpy()

TypeError: 'ParameterDict' object is not callable

In [104]:
test = my_net.collect_params()

In [105]:
test

medico_task (
  Parameter medico_taskconv1 (shape=None, dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm0_gamma (shape=(3,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm0_beta (shape=(3,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm0_running_mean (shape=(3,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm0_running_var (shape=(3,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskconv0_weight (shape=(64, 3, 7, 7), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm1_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm1_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm1_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskbatchnorm1_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskstage1_batchnorm0_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter medico_taskstage1_ba

Parameter medico_taskconv1 (shape=None, dtype=<class 'numpy.float32'>)

In [84]:
test = mx.sym.deepcopy()

AttributeError: module 'mxnet.symbol' has no attribute 'deepcopy'

In [76]:
out_acc, out_loss  = evaluate_accuracy(dataloader_test, softmax_cross_entropy, my_net)
print("Untrained network Test Accuracy: {0:.4f} Test loss: {1:.4f}"
      .format(out_acc, out_loss.asscalar()))

cumulative loss = 
[344.77707]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Untrained network Test Accuracy: 0.9267 Test loss: 0.2160


In [56]:
out_acc, out_loss  = evaluate_accuracy(dataloader_test, softmax_cross_entropy, my_net)
print("Untrained network Test Accuracy: {0:.4f} Test loss: {1:.4f}"
      .format(out_acc, out_loss))

cumulative loss = 465.36513662338257 no_of_samples = 1596
Untrained network Test Accuracy: 0.9123 Test loss: 0.2916


In [43]:
test_array_gpu = mx.nd.empty((3,2), ctx=ctx)

In [44]:
test_array_gpu


[[0.    1.875]
 [0.    0.   ]
 [0.    0.   ]]
<NDArray 3x2 @gpu(0)>

In [45]:
test_array_gpu[0,0], test_array_gpu[0, 1] = evaluate_accuracy(dataloader_test, softmax_cross_entropy, my_net)

cumulative loss = 751.9391508102417 no_of_samples = 1596


In [46]:
test_array_gpu


[[0.8784461  0.47113982]
 [0.         0.        ]
 [0.         0.        ]]
<NDArray 3x2 @gpu(0)>

## Trainer

In [77]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [78]:
LEARNING_RATE = 0.0005
WDECAY = 0.00001
MOMENTUM = 0.9

In [79]:
trainer = gluon.Trainer(my_net.collect_params(), 'sgd', 
                        {'learning_rate': LEARNING_RATE,
                         'wd':WDECAY,
                         'momentum':MOMENTUM})

In [80]:
model_dir = data_dir + '/mxnet_models'
model_name = "test_mxnet_model_v2_"

In [112]:
def train_model(net, trainer, loss_fn, num_epochs = 1):
    

    #val_accuracy = 0
    val_accuracy = mx.nd.zeros(1, ctx=ctx)
   # df = pd.DataFrame(columns=['train_acc', 'train_loss', 'val_acc', 'val_loss'])
    history = mx.nd.empty((num_epochs, 4), ctx=ctx) # 4 represents = train_acc, train_loss, val_acc, val_loss


    for epoch in range(num_epochs):
        for i, (data, label) in enumerate(dataloader_train):
            data = data.astype(np.float32).as_in_context(ctx)
            label = label.as_in_context(ctx)

            if i%20==0 and i >0:
                print('Batch [{0}] loss: {1:.4f}'
                      .format(i, loss.mean().asscalar()))

            with autograd.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)
                loss.backward()
            trainer.step(data.shape[0])
            
            
        nd.waitall() # wait at the end of the epoch    


      #  train_accuracy, train_loss = evaluate_accuracy(dataloader_train, loss_fn, net)
        history[epoch,0], history[epoch,1] = evaluate_accuracy(dataloader_train, loss_fn, net)
       # new_val_accuracy, new_val_loss = evaluate_accuracy(dataloader_test, loss_fn, net)
        history[epoch,2], history[epoch,3] = evaluate_accuracy(dataloader_test, loss_fn, net)
        new_val_accuracy = history[epoch,2]
    #    df2 = pd.DataFrame([[train_accuracy, train_loss, new_val_accuracy, new_val_loss]], 
     #                      columns=['train_acc', 'train_loss', 'val_acc', 'val_loss'])
        #new_val_accuracy = evaluate_accuracy_gluon(dataloader_test, my_net) 
      #  print("all done")
     #   print(type(train_accuracy))
     #   print("Epoch [{0}] Train accuracy {1:.4f} val Accuracy {2:.4f} " 
     #        .format(epoch, train_accuracy, new_val_accuracy))
     #   print("Epoch [{0}] Train loss {1:.4f} val loss {2:.4f} " 
     #         .format(epoch, train_loss, new_val_loss))

        # We perform early-stopping regularization, to prevent the model from overfitting
   #     df = df.append(df2, ignore_index=True)
        if new_val_accuracy > val_accuracy:
            print('Validation accuracy is increasing.. saving the model')
            best_model_name_temp = model_name + "deep_copy" + str(epoch)
            
           # my_net.save_parameters(os.path.join(model_dir, model_name_temp))
            best_model = copy.deepcopy(net)
            #break
            val_accuracy = new_val_accuracy
    
    # at last: save the best model to HDD
    best_model.save_parameters(os.path.join(model_dir, best_model_name_temp))
    
    return history



In [62]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0323
Batch [40] loss: 0.0773
Batch [60] loss: 0.0639
Batch [80] loss: 0.1261
Batch [100] loss: 0.0341
cumulative loss = 60.652903348207474 no_of_samples = 3697
cumulative loss = 60.65290443599224 no_of_samples = 3697
cumulative loss = 347.2140165567398 no_of_samples = 1596
cumulative loss = 347.2140169143677 no_of_samples = 1596
all done
<class 'numpy.float64'>
Epoch [0] Train accuracy 0.9986 val Accuracy 0.9229 
Epoch [0] Train loss 0.0164 val loss 0.2176 
Validation accuracy is increasing.. saving the model
CPU times: user 51.6 s, sys: 21.3 s, total: 1min 12s
Wall time: 1min 9s


(   train_acc  train_loss   val_acc  val_loss
 0   0.998648    0.016406  0.922932  0.217553, 
 [[0.9986476  0.01640598 0.9229323  0.21755265]]
 <NDArray 1x4 @gpu(0)>)

In [66]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0248
Batch [40] loss: 0.0293
Batch [60] loss: 0.0510
Batch [80] loss: 0.1492
Batch [100] loss: 0.0387
cumulative loss = 31.505593672394753 no_of_samples = 3697
cumulative loss = 344.77702701091766 no_of_samples = 1596
Validation accuracy is increasing.. saving the model
CPU times: user 38.2 s, sys: 16.1 s, total: 54.3 s
Wall time: 50.3 s



[[0.99918854 0.00852193 0.9266917  0.21602571]]
<NDArray 1x4 @gpu(0)>

In [82]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0163
Batch [40] loss: 0.0115
Batch [60] loss: 0.0282
Batch [80] loss: 0.0239
Batch [100] loss: 0.0122
cumulative loss = 
[25.468235]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[355.90643]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 38.3 s, sys: 16.3 s, total: 54.6 s
Wall time: 50.7 s



[[0.99918854 0.00688889 0.92418545 0.22299902]]
<NDArray 1x4 @gpu(0)>

In [107]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0087
Batch [40] loss: 0.0131
Batch [60] loss: 0.0138
Batch [80] loss: 0.0436
Batch [100] loss: 0.0237
cumulative loss = 
[22.111965]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[360.91272]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 38 s, sys: 15.6 s, total: 53.6 s
Wall time: 50.1 s



[[0.999459   0.00598106 0.9235589  0.22613579]]
<NDArray 1x4 @gpu(0)>

In [109]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0104
Batch [40] loss: 0.0103
Batch [60] loss: 0.0273
Batch [80] loss: 0.0109
Batch [100] loss: 0.0140
cumulative loss = 
[16.213184]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[356.77686]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 38.4 s, sys: 16.1 s, total: 54.5 s
Wall time: 50.8 s



[[0.9997295  0.0043855  0.9254386  0.22354439]]
<NDArray 1x4 @gpu(0)>

In [111]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0088
Batch [40] loss: 0.0087
Batch [60] loss: 0.0144
Batch [80] loss: 0.0325
Batch [100] loss: 0.0091
cumulative loss = 
[13.436509]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[351.14673]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 38.8 s, sys: 17.5 s, total: 56.3 s
Wall time: 52.4 s



[[0.9997295  0.00363444 0.9285714  0.22001675]]
<NDArray 1x4 @gpu(0)>

In [113]:
%%time
train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0072
Batch [40] loss: 0.0074
Batch [60] loss: 0.0072
Batch [80] loss: 0.0119
Batch [100] loss: 0.0856
cumulative loss = 
[12.45869]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[356.69644]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 39.6 s, sys: 29.5 s, total: 1min 9s
Wall time: 1min 6s



[[0.9997295  0.00336995 0.9273183  0.22349401]]
<NDArray 1x4 @gpu(0)>

In [114]:
%%time
history = train_model(my_net, trainer=trainer, loss_fn=softmax_cross_entropy, num_epochs=NUM_OF_EPOCHS)

Batch [20] loss: 0.0152
Batch [40] loss: 0.0109
Batch [60] loss: 0.0037
Batch [80] loss: 0.0080
Batch [100] loss: 0.0055
cumulative loss = 
[11.954583]
<NDArray 1 @gpu(0)> no_of_samples = 
[3697.]
<NDArray 1 @gpu(0)>
cumulative loss = 
[353.07568]
<NDArray 1 @gpu(0)> no_of_samples = 
[1596.]
<NDArray 1 @gpu(0)>
Validation accuracy is increasing.. saving the model
CPU times: user 40 s, sys: 30.4 s, total: 1min 10s
Wall time: 1min 7s


In [115]:
history


[[0.9997295  0.00323359 0.92418545 0.22122537]]
<NDArray 1x4 @gpu(0)>

In [118]:
history_cpu = history.as_in_context(mx.cpu()).asnumpy()

In [119]:
history_cpu

array([[0.9997295 , 0.00323359, 0.92418545, 0.22122537]], dtype=float32)

In [ ]:
df

In [ ]:
df.plot()

## Saving

In [ ]:
model_dir = data_dir + '/mxnet_models'

In [ ]:
model_name = "test_mxnet_model_v1_96"

In [ ]:
my_net.save_parameters(os.path.join(model_dir, model_name))

## Visualizing trainign and validation accuracy and loss

In [ ]:
my_net.

## Loading the saved models

In [ ]:
my_net.load_parameters(os.path.join(model_dir, model_name))

## Testing

In [ ]:
true_labels = mx.nd.array([2])

In [ ]:
true_labels

In [ ]:
temp = mx.nd.array([1,2,3])

In [ ]:
temp

In [ ]:
true_labels = mx.ndarray.concat(true_labels, temp, dim=0)

In [ ]:
true_labels.asnumpy()

In [ ]:
temp

In [ ]:
acc = mx.metric.Accuracy()

all_true_labels = np.array([])#mx.nd.array([0])
#temp = mx.nd.array([1,2,3,4,5])
#all_predicted_labels = []
all_predicted_labels_device = mx.nd.array([-1], ctx=ctx) # mx.nd


for i,(data, label) in enumerate(dataloader_test):
    
    data = data.astype(np.float32).as_in_context(ctx) # loading data to GPU if available
    l = label.asnumpy()
    # label = label #.as_in_context(ctx) # loading data to GPU if available
   # all_true_labels = mx.ndarray.concat(all_true_labels,l, dim=0 )
    all_true_labels = np.concatenate((all_true_labels,l))
    print(l)
    print("====")
    print(label)
    print("====")
    print(len(all_true_labels))
    
    with autograd.predict_mode():
        probability=my_net(data)
        predictions = nd.argmax(probability, axis=1)
        all_predicted_labels_device = mx.ndarray.concat(all_predicted_labels_device, predictions, dim=0)
        print(predictions)
        acc.update(preds=predictions, labels=label)
      #  print(acc.get()[1])
        #all_true_labels.extend(label)
       
        #all_predicted_labels.extend(predictions)
        print("gpu array =",all_predicted_labels_device)
      #  print(label)
       

    

In [ ]:
all_true_labels


In [ ]:
all_predicted_labels_device

In [ ]:
all_predicted_labels_cpu = all_predicted_labels_device.as_in_context(mx.cpu())

In [ ]:
all_predicted_labels_cpu = all_predicted_labels_cpu[1:].asnumpy()

In [ ]:
all_predicted_labels_cpu

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
all_true_labels

In [ ]:
#  Generate confusion matrix
cm = confusion_matrix(all_true_labels, all_predicted_labels_cpu)

In [ ]:
cm

In [ ]:
# for i, (data, label) in enumerate(dataloader_t):
 # with autograd.predict_mode():
    

In [ ]:
def plot_training_history(history, dir, plt_name):
    train_acc = history.history['acc']
    val_acc = history.history['val_acc']
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    df = pd.DataFrame({'train_acc':train_acc, 'train_loss':train_loss, 'val_acc':val_acc, 'val_loss':val_loss})
    pie = df.plot()
    fig = pie.get_figure()
    #fig.savefig(os.path.join(plot_dir, acc_loss_plot_name))
    fig.savefig(os.path.join(dir, plt_name))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          plt_size=[10,10]):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.rcParams['figure.figsize'] = plt_size
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig(os.path.join(plot_dir, cm_plot_name))

In [ ]:
plot_dir  = data_dir + '/mxnet_plots'

In [ ]:
cm_plot_name = 'cm_'+model_name

In [ ]:
plot_confusion_matrix(cm, classes=categories, title='my confusion matrix')